In [1]:
#原來一般的WebDriver
from selenium import webdriver
from selenium.webdriver import chrome
from selenium.webdriver import firefox
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import time
from queue import Queue
import codecs
from threading import Thread
import threading
from urllib.request import urlopen
import json
import os
import random
import csv
WINDOW_SIZE = "1000,1000"   #窗体大小
EDGEDRIVER_PATH=r"C:/Users/HP/microsoft_webdriver/MicrosoftWebDriver.exe"   # edge_driver 路徑
FIREFOXDRIVER_PATH=r"C:/Users/HP/geckodriver/geckodriver.exe"    # firefox_driver 路徑
FIREFOX_PATH=r"C:/Program Files/Mozilla Firefox/firefox.exe"     # firefox 路徑
firefox_options = firefox.options.Options()
firefox_options.add_argument("--headless")
firefox_options.add_argument("--window-size=%s" % WINDOW_SIZE)
firefox_options.binary_location = FIREFOX_PATH
firefox_profile = webdriver.FirefoxProfile()
firefox_profile.set_preference("dom.webnotifications.enabled", False)    # firefox disable notification

CHROME_PATH = r"C:/Program Files (x86)/Google/Chrome/Application/chrome.exe"  # chrome 路径
CHROMEDRIVER_PATH = r"C:/Users/HP/chromedriver_win32/chromedriver.exe" # chrome_driver 路径  
chrome_options = chrome.options.Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--incognito") #無痕模式
#禁止圖片加載跟跳通知
prefs = {"profile.default_content_setting_values.notifications" : 2, "profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
chrome_options.binary_location = CHROME_PATH

In [2]:
#換成用webdriver的方式進入網頁找
#class用.
#id用
def search(driver, keyword):
    see=driver.find_element_by_css_selector("._1frb")
    see.send_keys(keyword)
    go=driver.find_element_by_css_selector("._585_")
    go.submit()
    time.sleep(3)
    driver.save_screenshot(keyword+"result.png")
#處理捲動式頁面
#以卷動到無更多搜尋結果
    see_all=driver.find_elements_by_class_name("_5w9f")
    driver.get(see_all[1].get_attribute("href"))
    time.sleep(1)
    driver.save_screenshot("see_"+keyword+"all.png")
    t=driver.find_elements_by_css_selector('[class="phm _64f"]')
    while len(t) is 0:
        #ActionChains(driver).key_down(Keys.PAGE_DOWN).perform()
        #driver.find_element_by_css_selector("html").send_keys(Keys.PAGE_DOWN)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5)
        t=driver.find_elements_by_css_selector('[class="phm _64f"]')
    driver.save_screenshot("search_"+keyword+"results.png")
    di=driver.find_elements_by_css_selector('[class="_3u1 _gli _6pe1"]')
    name=driver.find_elements_by_css_selector("._32mo")
    idid=[]
    for i in di:
        p=i.get_attribute("data-bt").replace("null","0").replace("false", "0")
        idid.append(eval(p)["id"])
#name_url是一個list 存放有每個人的人名及url
    nameurl = []
    for h, j in zip(name,idid):
        nameurl.append(h.text)
        nameurl.append(h.get_attribute("href"))
        nameurl.append(j)
        print(nameurl)
#將所有的名子存成檔案
    namelist=codecs.open(keyword+"namelist.txt","w+","utf-8")
    for i in nameurl:
        if type(i)==int:
            i=str(i)
        namelist.write(i)
        namelist.write("\n")
    namelist.close()
    return 

In [4]:
#用已經存好了的name_url開始一個個找
def information(driver, filename, email0, password0, email1, password1, email2, password2, email3, password3):
#開啟名子檔案
    name=codecs.open(filename+"namelist.txt","r+","utf-8")
    name_url=name.read().split("\n")
    del name_url[len(name_url)-1]
    name.close()
#開啟一份爬找失敗的url
    fail=[]
    rn=random.randrange(15,21, 3)
    data=codecs.open(filename+"_data.json", "w+")
    fail_file=codecs.open(filename+"fail.txt","w+","utf-8")
    for i in range(0, len(name_url), rn):
        d=i/rn
        if d%4==0: 
            browser="chrome"
            driver= webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options)
            driver.get("https://www.facebook.com/")
            time.sleep(5)
            user=driver.find_element_by_css_selector("#email")
            user.send_keys(email0)
            time.sleep(1)
            pwd=driver.find_element_by_css_selector("#pass")
            pwd.send_keys(password0)
            time.sleep(1)
            log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
            log.submit()
            time.sleep(5)
            driver.save_screenshot("screenshot_"+browser+".png")
        elif d%4==1:
            try:
                browser="firefox"
                driver=webdriver.Firefox(executable_path=FIREFOXDRIVER_PATH, firefox_options=firefox_options, firefox_profile=firefox_profile)
                driver.get("https://www.facebook.com/")
                time.sleep(5)
                user=driver.find_element_by_css_selector("#email")
                user.send_keys(email1)
                time.sleep(1)
                pwd=driver.find_element_by_css_selector("#pass")
                pwd.send_keys(password1)
                time.sleep(1)
                log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
                log.submit()
                time.sleep(5)
                driver.save_screenshot("screenshot_"+browser+"1.png")
            except FileNotFoundError:
                browser="chrome"
                driver= webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options)
                driver.get("https://www.facebook.com/")
                time.sleep(5)
                user=driver.find_element_by_css_selector("#email")
                user.send_keys(email1)
                time.sleep(1)
                pwd=driver.find_element_by_css_selector("#pass")
                pwd.send_keys(password1)
                time.sleep(1)
                log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
                log.submit()
                time.sleep(5)
                driver.save_screenshot("screenshot_"+browser+"1.png")
        elif d%4==2:
            browser="chrome"
            driver=webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options)
            driver.get("https://www.facebook.com/")
            time.sleep(5)
            user=driver.find_element_by_css_selector("#email")
            user.send_keys(email2)
            time.sleep(1)
            pwd=driver.find_element_by_css_selector("#pass")
            pwd.send_keys(password2)
            time.sleep(1)
            log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
            log.submit()
            time.sleep(5)
            driver.save_screenshot("screenshot_"+browser+"2.png")
        elif d%4==3:
            try:
                browser="firefox"
                driver=webdriver.Firefox(executable_path=FIREFOXDRIVER_PATH, firefox_options=firefox_options, firefox_profile=firefox_profile)
                driver.get("https://www.facebook.com/")
                time.sleep(5)
                user=driver.find_element_by_css_selector("#email")
                user.send_keys(email3)
                time.sleep(1)
                pwd=driver.find_element_by_css_selector("#pass")
                pwd.send_keys(password3)
                time.sleep(1)
                log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
                log.submit()
                time.sleep(5)
                driver.save_screenshot("screenshot_"+browser+"3.png")
            except FileNotFoundError:
                browser="chrome"
                driver= webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options)
                driver.get("https://www.facebook.com/")
                time.sleep(5)
                user=driver.find_element_by_css_selector("#email")
                user.send_keys(email3)
                time.sleep(1)
                pwd=driver.find_element_by_css_selector("#pass")
                pwd.send_keys(password3)
                time.sleep(1)
                log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
                log.submit()
                time.sleep(5)
                driver.save_screenshot("screenshot_"+browser+"3.png")
        for n in range(i, i+rn):
            if n%3==1:
                info={}
                print(name_url[n])
                info[name_url[n+1]]={}
                driver.get(name_url[n])
                time.sleep(random.uniform(4, 8))
                file = "Screenshot_for_Facebook_"+filename
                if not os.path.exists(file):    #先確認資料夾是否存在
                    os.makedirs(file)
                driver.save_screenshot("Screenshot_for_Facebook_"+filename+"/profile--" + str(n) + ".png")
                try:
                    rel=driver.find_element_by_link_text("關於")
                    driver.get(rel.get_attribute("href"))
                    
                except Exception:
                    print("---------操作過快-----------")
                    fail.append(name_url[n-1])
                    fail.append(name_url[n])
                    fail.append(name_url[n+1])
                    fail_file.write(name_url[n])
                    fail_file.write("\n")
                time.sleep(random.uniform(2, 5))
#profile中--總攬
    #overview--list
                overview=[name_url[n-1]]
                for k in driver.find_elements_by_xpath('//div[@class="_c24 _50f4"]'):
                    overview.append(k.text.replace("\n",":"))
                for j in driver.find_elements_by_xpath('//span[@class="_c24 _2ieq"]'):
                    overview.append(j.text.replace("\n", ":"))
                info[name_url[n+1]]["overview"] = overview
#學歷及工作經驗
    #edu_work--list
                edu_work=[]
                try:
                    edu=driver.find_element_by_link_text("學歷與工作經歷")
                    driver.get(edu.get_attribute("href"))
                    try:
                        for e in driver.find_elements_by_css_selector('[class="_2lzr _50f5 _50f7"]'):
                            edu_work.append(e.text)
                    except Exception:
                        pass
                except Exception:
                    print("---------操作過快-----------")
                    if name_url[n] not in fail:
                        fail.append(name_url[n-1])
                        fail.append(name_url[n])
                        fail.append(name_url[n+1])
                        fail_file.write(name_url[n])
                        fail_file.write("\n")
                info[name_url[n+1]]["edu_work"] = edu_work
                time.sleep(random.uniform(2, 5))
                '''
                for e in driver.find_elements_by_xpath('//div[@class="_6a _6b"]'):
                    edu_work.append(e.text)
                
                try:
                    for _ in range(len(edu_work)):
                        edu_work.remove("")
                except ValueError:
                    info[name_url[n+1]]["edu_work"] = edu_work
                '''
#居住地點
    #place--list
                addr=[]
                try:
                    living=driver.find_element_by_xpath('//a[@class="_5pwr _Interaction__ProfileSectionPlaces"]')
                    driver.get(living.get_attribute("href"))
                    for p in driver.find_elements_by_xpath('//span[@class="_2iel _50f7"]'):
                        addr.append(p.text)
                except Exception:
                    print("---------操作過快-----------")
                    if name_url[n] not in fail:
                        fail.append(name_url[n-1])
                        fail.append(name_url[n])
                        fail.append(name_url[n+1])
                        fail_file.write(name_url[n])
                        fail_file.write("\n")
                info[name_url[n+1]]["addr"] = addr
                time.sleep(random.uniform(2, 5))
#聯絡資料
                contact=[]
                try:
                    cont=driver.find_element_by_xpath('//a[@class="_5pwr _Interaction__ProfileSectionContactBasic"]')
                    driver.get(cont.get_attribute("href"))
    #只有電話的部分
    #contact--list
                    try:
                        tel=driver.find_element_by_xpath('//span[@dir="ltr"]')
                        time.sleep(random.uniform(2, 5))
                        contact.append(tel.text)
                    except Exception:
                        contact.append("No phone")
                    try:
                        birth=driver.find_element_by_xpath('//*[@id="pagelet_basic"]/div/ul/li[1]/div/div[2]/div/div/span')
                        contact.append(birth.text.replace("\n",":"))
                        time.sleep(random.uniform(2, 5))
                    except Exception:
                        contact.append("No birth")
                    try:
                        gender=driver.find_element_by_css_selector('[class="_3pw9 _2pi4 _2ge8 _3ms8"]')
                        contact.append(gender.text.replace("\n",":"))
                        time.sleep(random.uniform(2, 5))
                    except Exception:
                        contact.append("No gender")
                except Exception:
                    print("---------操作過快-----------")
                    if name_url[n] not in fail:
                        fail.append(name_url[n-1])
                        fail.append(name_url[n])
                        fail.append(name_url[n+1])
                        fail_file.write(name_url[n])
                        fail_file.write("\n")
                info[name_url[n+1]]["contact"] = contact
                time.sleep(random.uniform(2, 5))
#家人與感情狀況
                relation={}
                try:
                    rel=driver.find_element_by_xpath('//a[@class="_5pwr _Interaction__ProfileSectionAllRelationships"]')
                    driver.get(rel.get_attribute("href"))
#這些人的姓名及ID
    #relation--dict
                    try:    #感情
                        for r in driver.find_elements_by_css_selector('[class="_2lzr _50f5 _50f7"]'):
                            relation[r.text]=r.get_attribute('data-hovercard').split("=")[1]
                    except Exception:
                        pass                        #class="_50f8 _2iem" 家人跟感情裝況 無資料都是這個
                    try:    #家人
                        for w in driver.find_elements_by_css_selector('[class="_2iel _50f7"]'):
                            relation[w.text]=w.get_attribute('data-hovercard').split("=")[1]
                    except Exception:
                        pass                        #class="_50f8 _2iem" 家人跟感情裝況 無資料都是這個
                    '''
                    for r in driver.find_elements_by_xpath('//a[@data-hovercard]'):
                        relation[r.text]=r.get_attribute('data-hovercard')[28:]
                    info[name_url[n+1]]["family"] = relation
                    '''
                    #class="_2iel _50f7" 家人
                    #class="_2lzr _50f5 _50f7" 感情狀況
                except Exception:
                    print("---------操作過快-----------")
                    if name_url[n] not in fail:
                        fail.append(name_url[n-1])
                        fail.append(name_url[n])
                        fail.append(name_url[n+1])
                        fail_file.write(name_url[n])
                        fail_file.write("\n")
                info[name_url[n+1]]["family"]=relation
                time.sleep(random.uniform(2, 5))

#詳細資料--看看會不會有一些話可以判斷他的行為
                try:
                    words=driver.find_element_by_xpath('//a[@class="_5pwr _Interaction__ProfileSectionAbout"]')
                    driver.get(words.get_attribute("href"))
    #about=driver.find_element_by_xpath('//*[@id="pagelet_timeline_app_collection_100000446869156:2327158227:8"]/ul/li/div/div[1]/div/div/ul/li[6]/a')
    #about--list
                    try:
                        q = driver.find_elements_by_css_selector("._2iem")
                        for b in q:
                            info[name_url[n+1]]["details"] = b.text
                    except:
                        a = driver.find_element_by_xpath('//span[@class="_50f8 _2iem"]')
                        info[name_url[n+1]]["details"] = a.text
                    try:
                        w = driver.find_element_by_xpath("//*[@id='pagelet_quotes']/div/ul/li/div/div/span")
                        time.sleep(random.uniform(2, 5))
                        info[name_url[n+1]]["details"] = w.text
                    except:
                        u = driver.find_elements_by_css_selector("._50f8 _2iem")
                        if len(u)>1:
                            info[name_url[n+1]]["details"] = u[1].text
                except Exception:
                    print("---------操作過快-----------")
                    if name_url[n] not in fail:
                        fail.append(name_url[n-1])
                        fail.append(name_url[n])
                        fail.append(name_url[n+1])
                        fail_file.write(name_url[n])
                        fail_file.write("\n")
                time.sleep(random.uniform(2, 5))
#生活要事
    #timeline--list
                timeline=[]
                try:
                    life=driver.find_element_by_xpath('//a[@class="_5pwr _Interaction__ProfileSectionYearOverviews"]')
                    driver.get(life.get_attribute("href"))
                    line=driver.find_elements_by_xpath('//div[@class="clearfix _ikh"]')
                    for t in line:
                        timeline.append(t.text.replace("\n",":"))
                except Exception:
                    print("---------操作過快-----------")
                    if name_url[n] not in fail:
                        fail.append(name_url[n-1])
                        fail.append(name_url[n])
                        fail.append(name_url[n+1])
                        fail_file.write(name_url[n])
                        fail_file.write("\n")
                info[name_url[n+1]]["timeline"] = timeline
                time.sleep(random.uniform(2, 5))
                json.dump(info, data)
                data.write('\n')
            else:
                if n<len(name_url):
                    print(name_url[n])
                else:
                    print("-----爬蟲結束----")
                    break #只要跳出這個迴圈就可，已經是最後了
#關掉目前所有的driver、即刪除cookie
        driver.delete_all_cookies()
#重新用其他帳號再次登入
#因為driver已經quit了，所以要重新定義一個新的driver才能再次啟動
        driver.quit()        
        time.sleep(3)
#將失敗的收集起來
    fail_file.close()
    data.close()
    return fail

In [17]:
#要再寫一個對於fail的爬蟲
#用已經存好了的name_url開始一個個找
def fail_info(driver, filename, email0, password0, email1, password1, email2, password2, email3, password3):
#開啟fail名子檔案
    name=codecs.open(filename+"fail.txt","r+","utf-8")
    name_url=name.read().split("\n")
    del name_url[len(name_url)-1]
    name.close()
    rn=random.randrange(15,24, 3)
    fail=[]
#開啟原有的data， 存成 info只要把fail的蓋掉就好
    info={}
    with open(filename+"_data.json","r+") as file:
        for line in file.readlines():
            dic = json.loads(line)
            for key, value in dic.items():
                info[key]=value
    data=codecs.open(filename+"_data.json", "w+") #只要重新存成一個info才重新寫入，就不會遺失掉覆蓋的
    fail_file=codecs.open(filename+"fail.txt","w+","utf-8")
    for i in range(0, len(name_url), rn):
        d=i/rn
        if d%4==0: 
            browser="chrome"
            driver= webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options)
            driver.get("https://www.facebook.com/")
            time.sleep(5)
            user=driver.find_element_by_css_selector("#email")
            user.send_keys(email0)
            time.sleep(1)
            pwd=driver.find_element_by_css_selector("#pass")
            pwd.send_keys(password0)
            time.sleep(1)
            log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
            log.submit()
            time.sleep(5)
            driver.save_screenshot("screenshot_"+browser+".png")
        elif d%4==1:
            try:
                browser="firefox"
                driver=webdriver.Firefox(executable_path=FIREFOXDRIVER_PATH, firefox_options=firefox_options, firefox_profile=firefox_profile)
                driver.get("https://www.facebook.com/")
                time.sleep(5)
                user=driver.find_element_by_css_selector("#email")
                user.send_keys(email1)
                time.sleep(1)
                pwd=driver.find_element_by_css_selector("#pass")
                pwd.send_keys(password1)
                time.sleep(1)
                log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
                log.submit()
                time.sleep(5)
                driver.save_screenshot(browser+"1.png")
            except FileNotFoundError:
                browser="chrome"
                driver= webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options)
                driver.get("https://www.facebook.com/")
                time.sleep(5)
                user=driver.find_element_by_css_selector("#email")
                user.send_keys(email1)
                time.sleep(1)
                pwd=driver.find_element_by_css_selector("#pass")
                pwd.send_keys(password1)
                time.sleep(1)
                log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
                log.submit()
                time.sleep(5)
                driver.save_screenshot(browser+"1.png")
        elif d%4==2:
            browser="chrome"
            driver=webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options)
            driver.get("https://www.facebook.com/")
            time.sleep(5)
            user=driver.find_element_by_css_selector("#email")
            user.send_keys(email2)
            time.sleep(1)
            pwd=driver.find_element_by_css_selector("#pass")
            pwd.send_keys(password2)
            time.sleep(1)
            log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
            log.submit()
            time.sleep(5)
            driver.save_screenshot(browser+"2.png")
        elif d%4==3:
            try:
                browser="firefox"
                driver=webdriver.Firefox(executable_path=FIREFOXDRIVER_PATH, firefox_options=firefox_options, firefox_profile=firefox_profile)
                driver.get("https://www.facebook.com/")
                time.sleep(5)
                user=driver.find_element_by_css_selector("#email")
                user.send_keys(email3)
                time.sleep(1)
                pwd=driver.find_element_by_css_selector("#pass")
                pwd.send_keys(password3)
                time.sleep(1)
                log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
                log.submit()
                time.sleep(5)
                driver.save_screenshot(browser+"3.png")
            except FileNotFoundError:
                browser="chrome"
                driver= webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options)
                driver.get("https://www.facebook.com/")
                time.sleep(5)
                user=driver.find_element_by_css_selector("#email")
                user.send_keys(email3)
                time.sleep(1)
                pwd=driver.find_element_by_css_selector("#pass")
                pwd.send_keys(password3)
                time.sleep(1)
                log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
                log.submit()
                time.sleep(5)
                driver.save_screenshot(browser+"3.png")
        for n in range(i, i+rn):
            if n%3==1:
                info={}
                print(name_url[n])
                info[name_url[n+1]]={}
                driver.get(name_url[n])
                time.sleep(random.uniform(4, 8))
                file = "Screenshot_for_Facebook_fail_"+filename
                if not os.path.exists(file):    #先確認資料夾是否存在
                    os.makedirs(file)
                driver.save_screenshot("Screenshot_for_Facebook_fail_"+filename+"/fail_profile--" + str(n) + ".png")
                try:
                    rel=driver.find_element_by_link_text("關於")
                    driver.get(rel.get_attribute("href"))
                except Exception:
                    print("---------操作過快-----------")
                    fail.append(name_url[n-1])
                    fail.append(name_url[n])
                    fail.append(name_url[n+1])
                    fail_file.write(name_url[n])
                    fail_file.write("\n")
                time.sleep(random.uniform(2, 5))
#profile中--總攬
    #overview--list
                overview=[name_url[n-1]]
                for k in driver.find_elements_by_xpath('//div[@class="_c24 _50f4"]'):
                    overview.append(k.text)
                for j in driver.find_elements_by_xpath('//span[@class="_c24 _2ieq"]'):
                    overview.append(j.text)
                info[name_url[n+1]]["overview"] = overview
#學歷及工作經驗
    #edu_work--list
                edu_work=[]
                try:
                    edu=driver.find_element_by_link_text("學歷與工作經歷")
                    driver.get(edu.get_attribute("href"))
                    try:
                        for e in driver.find_elements_by_css_selector('[class="_2lzr _50f5 _50f7"]'):
                            edu_work.append(e.text)
                    except Exception:
                        pass
                except Exception:
                    print("---------操作過快-----------")
                    if name_url[n] not in fail:
                        fail.append(name_url[n-1])
                        fail.append(name_url[n])
                        fail.append(name_url[n+1])
                        fail_file.write(name_url[n])
                        fail_file.write("\n")
                info[name_url[n+1]]["edu_work"] = edu_work
                time.sleep(random.uniform(2, 5))
                '''
                for e in driver.find_elements_by_xpath('//div[@class="_6a _6b"]'):
                    edu_work.append(e.text)
                
                try:
                    for _ in range(len(edu_work)):
                        edu_work.remove("")
                except ValueError:
                    info[name_url[n+1]]["edu_work"] = edu_work
                '''
#居住地點
    #place--list
                addr=[]
                try:
                    living=driver.find_element_by_xpath('//a[@class="_5pwr _Interaction__ProfileSectionPlaces"]')
                    driver.get(living.get_attribute("href"))
                    for p in driver.find_elements_by_xpath('//span[@class="_2iel _50f7"]'):
                        addr.append(p.text)
                except Exception:
                    print("---------操作過快-----------")
                    if name_url[n] not in fail:
                        fail.append(name_url[n-1])
                        fail.append(name_url[n])
                        fail.append(name_url[n+1])
                        fail_file.write(name_url[n])
                        fail_file.write("\n")
                info[name_url[n+1]]["addr"] = addr
                time.sleep(random.uniform(2, 5))
#聯絡資料
                contact=[]
                try:
                    cont=driver.find_element_by_xpath('//a[@class="_5pwr _Interaction__ProfileSectionContactBasic"]')
                    driver.get(cont.get_attribute("href"))
    #只有電話的部分
    #contact--list
                    try:
                        tel=driver.find_element_by_xpath('//span[@dir="ltr"]')
                        time.sleep(random.uniform(2, 5))
                        contact.append(tel.text)
                    except Exception:
                        contact.append("No phone")
                    try:
                        birth=driver.find_element_by_xpath('//*[@id="pagelet_basic"]/div/ul/li[1]/div/div[2]/div/div/span')
                        contact.append(birth.text)
                        time.sleep(random.uniform(2, 5))
                    except Exception:
                        contact.append("No birth")
                    try:
                        gender=driver.find_element_by_css_selector('[class="_3pw9 _2pi4 _2ge8 _3ms8"]')
                        contact.append(gender.text)
                        time.sleep(random.uniform(2, 5))
                    except Exception:
                        contact.append("No gender")
                except Exception:
                    print("---------操作過快-----------")
                    if name_url[n] not in fail:
                        fail.append(name_url[n-1])
                        fail.append(name_url[n])
                        fail.append(name_url[n+1])
                        fail_file.write(name_url[n])
                        fail_file.write("\n")
                info[name_url[n+1]]["contact"] = contact
                time.sleep(random.uniform(2, 5))
#家人與感情狀況
                relation={}
                try:
                    rel=driver.find_element_by_xpath('//a[@class="_5pwr _Interaction__ProfileSectionAllRelationships"]')
                    driver.get(rel.get_attribute("href"))
#這些人的姓名及ID
    #relation--dict
                    try:    #感情
                        for r in driver.find_elements_by_css_selector('[class="_2lzr _50f5 _50f7"]'):
                            relation[r.text]=r.get_attribute('data-hovercard').split("=")[1]
                    except Exception:
                        pass                        #class="_50f8 _2iem" 家人跟感情裝況 無資料都是這個
                    try:    #家人
                        for w in driver.find_elements_by_css_selector('[class="_2iel _50f7"]'):
                            relation[w.text]=w.get_attribute('data-hovercard').split("=")[1]
                    except Exception:
                        pass                        #class="_50f8 _2iem" 家人跟感情裝況 無資料都是這個
                    '''
                    for r in driver.find_elements_by_xpath('//a[@data-hovercard]'):
                        relation[r.text]=r.get_attribute('data-hovercard')[28:]
                    info[name_url[n+1]]["family"] = relation
                    '''
                    
                    #class="_2iel _50f7" 家人
                    #class="_2lzr _50f5 _50f7" 感情狀況
                except Exception:
                    print("---------操作過快-----------")
                    if name_url[n] not in fail:
                        fail.append(name_url[n-1])
                        fail.append(name_url[n])
                        fail.append(name_url[n+1])
                        fail_file.write(name_url[n])
                        fail_file.write("\n")
                info[name_url[n+1]]["family"]=relation
                time.sleep(random.uniform(2, 5))

#詳細資料--看看會不會有一些話可以判斷他的行為
                try:
                    words=driver.find_element_by_xpath('//a[@class="_5pwr _Interaction__ProfileSectionAbout"]')
                    driver.get(words.get_attribute("href"))
    #about=driver.find_element_by_xpath('//*[@id="pagelet_timeline_app_collection_100000446869156:2327158227:8"]/ul/li/div/div[1]/div/div/ul/li[6]/a')
    #about--list
                    try:
                        q = driver.find_elements_by_css_selector("._2iem")
                        for b in q:
                            info[name_url[n+1]]["details"] = b.text
                    except:
                        a = driver.find_element_by_xpath('//span[@class="_50f8 _2iem"]')
                        info[name_url[n+1]]["details"] = a.text
                    try:
                        w = driver.find_element_by_xpath("//*[@id='pagelet_quotes']/div/ul/li/div/div/span")
                        time.sleep(random.uniform(2, 5))
                        info[name_url[n+1]]["details"] = w.text
                    except:
                        u = driver.find_elements_by_css_selector("._50f8 _2iem")
                        if len(u)>1:
                            info[name_url[n+1]]["details"] = u[1].text
                except Exception:
                    print("---------操作過快-----------")
                    if name_url[n] not in fail:
                        fail.append(name_url[n-1])
                        fail.append(name_url[n])
                        fail.append(name_url[n+1])
                        fail_file.write(name_url[n])
                        fail_file.write("\n")
                time.sleep(random.uniform(2, 5))
#生活要事
    #timeline--list
                timeline=[]
                try:
                    life=driver.find_element_by_xpath('//a[@class="_5pwr _Interaction__ProfileSectionYearOverviews"]')
                    driver.get(life.get_attribute("href"))
                    line=driver.find_elements_by_xpath('//div[@class="clearfix _ikh"]')
                    for t in line:
                        timeline.append(t.text)
                except Exception:
                    print("---------操作過快-----------")
                    if name_url[n] not in fail:
                        fail.append(name_url[n-1])
                        fail.append(name_url[n])
                        fail.append(name_url[n+1])
                        fail_file.write(name_url[n])
                        fail_file.write("\n")
                info[name_url[n+1]]["timeline"] = timeline
                time.sleep(random.uniform(2, 5))
                json.dump(info, data)
                data.write('\n')
                print(n)
            else:
                if n<len(name_url):
                    print(name_url[n])
                else:
                    print("-----爬蟲結束----")
                    break #只要跳出這個迴圈就好，已經是最後了
#關掉目前所有的driver、即刪除cookie
        driver.delete_all_cookies()
#重新用其他帳號再次登入
#因為driver已經quit了，所以要重新定義一個新的driver才能再次啟動
        driver.quit()        
        time.sleep(3)
#將失敗的收集起來
    fail_file.close()
    data.close()
    return fail

In [5]:
# chrome跟firefox是一個
def main_login(browser,email0,password0, email1, password1,email2, password2,email3, password3, keyword):
    '''如果哪天edge有headless的模式的話可用
    if browser == 'Edge':
        driver = webdriver.Edge(executable_path=EDGEDRIVER_PATH)
    '''
    if browser == 'Firefox':
        driver=webdriver.Firefox(executable_path=FIREFOXDRIVER_PATH, firefox_options=firefox_options, firefox_profile=firefox_profile)
    elif browser == 'Chrome':
        driver= webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options)
#先進到自己的facebook 才能查找之後的id
    driver.get("https://www.facebook.com/")
    driver.implicitly_wait(5)
    user=driver.find_element_by_css_selector("#email")
    user.send_keys(email2)
    time.sleep(1)
    pwd=driver.find_element_by_css_selector("#pass")
    pwd.send_keys(password2)
    time.sleep(1)
    log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
    log.submit()
    time.sleep(5)
#下面的screenshot可以確定是否有進入facebook
    driver.save_screenshot("screenshot_"+browser+".png")
    a=search(driver, keyword)
    #q.put(a)
    #lock.acquire()
    f=information(driver, keyword, email0, password0, email1, password1, email2, password2, email3, password3)
    time.sleep(3)
    #lock.release()
    return a

In [ ]:
def main_remedy(browser,email0,password0, email1, password1,email2, password2,email3, password3, keyword):
    '''如果哪天edge有headless的模式的話可用
    if browser == 'Edge':
        driver = webdriver.Edge(executable_path=EDGEDRIVER_PATH)
    '''
    if browser == 'Firefox':
        driver=webdriver.Firefox(executable_path=FIREFOXDRIVER_PATH, firefox_options=firefox_options, firefox_profile=firefox_profile)
    elif browser == 'Chrome':
        driver= webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options)
#先進到自己的facebook 才能查找之後的id
    driver.get("https://www.facebook.com/")
    driver.implicitly_wait(5)
    user=driver.find_element_by_css_selector("#email")
    user.send_keys(email2)
    time.sleep(1)
    pwd=driver.find_element_by_css_selector("#pass")
    pwd.send_keys(password2)
    time.sleep(1)
    log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
    log.submit()
    time.sleep(5)
    #q.put(a)
    #lock.acquire()
    f=fail_info(driver, keyword, email0, password0, email1, password1, email2, password2, email3, password3)
    time.sleep(3)
    #lock.release()
    return f

In [ ]:
#丟入要找的家族
target=["吳俊霖","吳俊樺","黃秋敏","黃秋米","吳俊融"]

#想要用的瀏覽器及輪換的帳號(目前設定四組)
browser="Chrome"
email0="123456@gmail.com"
password0="123"
email1="123456@yahoo.com.tw"
password1="123"
email2="123456789@yahoo.com.tw"
password2="123"

#將要找的目標個資全部抓下來並存檔
for keyword in target:
    a=main_login(browser,email0, password0, email1, password1,email2, password2,email3, password3, keyword)

In [32]:
#time.sleep("6小時的秒數 或是 換一組帳號重開")
#針對找失敗的用戶再次查詢更新資料
for keyword in target:
    f = main_remedy(browser,email0,password0, email1, password1,email2, password2,email3, password3, keyword)
    if len(f)!=0:
        print(keyword+"----還有一些用戶搜尋失敗，重跑一次這個部分")
    else:
        print(keyword+"----資料搜尋完成")

['黃秋米', 'https://www.facebook.com/profile.php?id=100027764510188&ref=br_rs', 100027764510188]
['黃秋米', 'https://www.facebook.com/profile.php?id=100027764510188&ref=br_rs', 100027764510188, '黃秋米', 'https://www.facebook.com/profile.php?id=100027749811265&ref=br_rs', 100027749811265]
['黃秋米', 'https://www.facebook.com/profile.php?id=100027764510188&ref=br_rs', 100027764510188, '黃秋米', 'https://www.facebook.com/profile.php?id=100027749811265&ref=br_rs', 100027749811265, '黃秋米', 'https://www.facebook.com/profile.php?id=100004514597547&ref=br_rs', 100004514597547]
['黃秋米', 'https://www.facebook.com/profile.php?id=100027764510188&ref=br_rs', 100027764510188, '黃秋米', 'https://www.facebook.com/profile.php?id=100027749811265&ref=br_rs', 100027749811265, '黃秋米', 'https://www.facebook.com/profile.php?id=100004514597547&ref=br_rs', 100004514597547, '黃秋米', 'https://www.facebook.com/profile.php?id=100013820209767&ref=br_rs', 100013820209767]
['黃秋米', 'https://www.facebook.com/profile.php?id=100027764510188&r

['黃秋米', 'https://www.facebook.com/profile.php?id=100027764510188&ref=br_rs', 100027764510188, '黃秋米', 'https://www.facebook.com/profile.php?id=100027749811265&ref=br_rs', 100027749811265, '黃秋米', 'https://www.facebook.com/profile.php?id=100004514597547&ref=br_rs', 100004514597547, '黃秋米', 'https://www.facebook.com/profile.php?id=100013820209767&ref=br_rs', 100013820209767, '黃秋米', 'https://www.facebook.com/profile.php?id=100026590274932&ref=br_rs', 100026590274932, '黃秋米', 'https://www.facebook.com/profile.php?id=100009921703324&ref=br_rs', 100009921703324, '黃秋米', 'https://www.facebook.com/profile.php?id=100013236857045&ref=br_rs', 100013236857045, '黃秋米', 'https://www.facebook.com/profile.php?id=100013956934114&ref=br_rs', 100013956934114, '黃秋蜜', 'https://www.facebook.com/profile.php?id=100010393930613&ref=br_rs', 100010393930613, '黃秋米', 'https://www.facebook.com/profile.php?id=100009229024938&ref=br_rs', 100009229024938, '黃秋米', 'https://www.facebook.com/profile.php?id=100022107619450&ref=b

['黃秋米', 'https://www.facebook.com/profile.php?id=100027764510188&ref=br_rs', 100027764510188, '黃秋米', 'https://www.facebook.com/profile.php?id=100027749811265&ref=br_rs', 100027749811265, '黃秋米', 'https://www.facebook.com/profile.php?id=100004514597547&ref=br_rs', 100004514597547, '黃秋米', 'https://www.facebook.com/profile.php?id=100013820209767&ref=br_rs', 100013820209767, '黃秋米', 'https://www.facebook.com/profile.php?id=100026590274932&ref=br_rs', 100026590274932, '黃秋米', 'https://www.facebook.com/profile.php?id=100009921703324&ref=br_rs', 100009921703324, '黃秋米', 'https://www.facebook.com/profile.php?id=100013236857045&ref=br_rs', 100013236857045, '黃秋米', 'https://www.facebook.com/profile.php?id=100013956934114&ref=br_rs', 100013956934114, '黃秋蜜', 'https://www.facebook.com/profile.php?id=100010393930613&ref=br_rs', 100010393930613, '黃秋米', 'https://www.facebook.com/profile.php?id=100009229024938&ref=br_rs', 100009229024938, '黃秋米', 'https://www.facebook.com/profile.php?id=100022107619450&ref=b

['黃秋米', 'https://www.facebook.com/profile.php?id=100027764510188&ref=br_rs', 100027764510188, '黃秋米', 'https://www.facebook.com/profile.php?id=100027749811265&ref=br_rs', 100027749811265, '黃秋米', 'https://www.facebook.com/profile.php?id=100004514597547&ref=br_rs', 100004514597547, '黃秋米', 'https://www.facebook.com/profile.php?id=100013820209767&ref=br_rs', 100013820209767, '黃秋米', 'https://www.facebook.com/profile.php?id=100026590274932&ref=br_rs', 100026590274932, '黃秋米', 'https://www.facebook.com/profile.php?id=100009921703324&ref=br_rs', 100009921703324, '黃秋米', 'https://www.facebook.com/profile.php?id=100013236857045&ref=br_rs', 100013236857045, '黃秋米', 'https://www.facebook.com/profile.php?id=100013956934114&ref=br_rs', 100013956934114, '黃秋蜜', 'https://www.facebook.com/profile.php?id=100010393930613&ref=br_rs', 100010393930613, '黃秋米', 'https://www.facebook.com/profile.php?id=100009229024938&ref=br_rs', 100009229024938, '黃秋米', 'https://www.facebook.com/profile.php?id=100022107619450&ref=b

['黃秋米', 'https://www.facebook.com/profile.php?id=100027764510188&ref=br_rs', 100027764510188, '黃秋米', 'https://www.facebook.com/profile.php?id=100027749811265&ref=br_rs', 100027749811265, '黃秋米', 'https://www.facebook.com/profile.php?id=100004514597547&ref=br_rs', 100004514597547, '黃秋米', 'https://www.facebook.com/profile.php?id=100013820209767&ref=br_rs', 100013820209767, '黃秋米', 'https://www.facebook.com/profile.php?id=100026590274932&ref=br_rs', 100026590274932, '黃秋米', 'https://www.facebook.com/profile.php?id=100009921703324&ref=br_rs', 100009921703324, '黃秋米', 'https://www.facebook.com/profile.php?id=100013236857045&ref=br_rs', 100013236857045, '黃秋米', 'https://www.facebook.com/profile.php?id=100013956934114&ref=br_rs', 100013956934114, '黃秋蜜', 'https://www.facebook.com/profile.php?id=100010393930613&ref=br_rs', 100010393930613, '黃秋米', 'https://www.facebook.com/profile.php?id=100009229024938&ref=br_rs', 100009229024938, '黃秋米', 'https://www.facebook.com/profile.php?id=100022107619450&ref=b

['黃秋米', 'https://www.facebook.com/profile.php?id=100027764510188&ref=br_rs', 100027764510188, '黃秋米', 'https://www.facebook.com/profile.php?id=100027749811265&ref=br_rs', 100027749811265, '黃秋米', 'https://www.facebook.com/profile.php?id=100004514597547&ref=br_rs', 100004514597547, '黃秋米', 'https://www.facebook.com/profile.php?id=100013820209767&ref=br_rs', 100013820209767, '黃秋米', 'https://www.facebook.com/profile.php?id=100026590274932&ref=br_rs', 100026590274932, '黃秋米', 'https://www.facebook.com/profile.php?id=100009921703324&ref=br_rs', 100009921703324, '黃秋米', 'https://www.facebook.com/profile.php?id=100013236857045&ref=br_rs', 100013236857045, '黃秋米', 'https://www.facebook.com/profile.php?id=100013956934114&ref=br_rs', 100013956934114, '黃秋蜜', 'https://www.facebook.com/profile.php?id=100010393930613&ref=br_rs', 100010393930613, '黃秋米', 'https://www.facebook.com/profile.php?id=100009229024938&ref=br_rs', 100009229024938, '黃秋米', 'https://www.facebook.com/profile.php?id=100022107619450&ref=b

黃秋米
https://www.facebook.com/profile.php?id=100027764510188&ref=br_rs
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
100027764510188
黃秋米
https://www.facebook.com/profile.php?id=100027749811265&ref=br_rs
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
100027749811265
黃秋米
https://www.facebook.com/profile.php?id=100004514597547&ref=br_rs
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
100004514597547
黃秋米
https://www.facebook.com/profile.php?id=100013820209767&ref=br_rs
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
---------操作過快-----------
---------操作過快---------

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//label[@class="uiButton uiButtonConfirm"]/input"}
  (Session info: headless chrome=68.0.3440.106)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)
